In [1]:
import sys
from pathlib import Path

sys.path.append(str(Path().resolve().parent / "src"))

project_root = Path().resolve().parent
data_path = project_root / "data" / "htad" / "features.csv"
file_path = project_root / "data" / "htad" / "user1" / "raw" / "1468360517106-acc-watch_tv.txt"
file_path2 = project_root / "data" / "htad" / "user3" / "raw" / "1471983192578-acc-brush_teeth.txt"

print("Exists?", file_path.exists())
print("Path:", file_path)

#acc_path = project_root / "data" / "htad" / "user1" / 

Exists? True
Path: /Users/johanneskeisers/Documents/GitHub/wearable-activity-htad/data/htad/user1/raw/1468360517106-acc-watch_tv.txt


In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from wearable_htad.data.preprocessing import load_features_csv
from wearable_htad.data.preprocessing import load_accelerometer_txt
from wearable_htad.data.preprocessing import load_all_accelerometer_files
from wearable_htad.features.pipeline import process_accelerometer_file

df = load_accelerometer_txt(file_path2)
#df = load_acceleromenter_txt("data/htad/raw/user1.txt")
df.head()



,real_timestamp,x,y,z,timestamp,sensor,activity
0,1471983193307,-0.171143,0.890869,0.449707,1471983192578,acc,brush_teeth
1,1471983193346,-0.210449,0.822266,0.454590,1471983192578,acc,brush_teeth
2,1471983193374,-0.228516,0.884033,0.474121,1471983192578,acc,brush_teeth
3,1471983193410,-0.248535,0.859375,0.466797,1471983192578,acc,brush_teeth
4,1471983193708,-0.146484,0.838135,0.532471,1471983192578,acc,brush_teeth


In [3]:
deltas = df["real_timestamp"].diff().dropna()
dt_ms = deltas.median()       # median gap in milliseconds
sampling_rate = 1000 / dt_ms  # convert to Hz

print(f"Median Δt: {dt_ms:.2f} ms")
print(f"Estimated Sampling Rate: {sampling_rate:.2f} Hz")


Median Δt: 34.00 ms
Estimated Sampling Rate: 29.41 Hz


In [8]:
from pathlib import Path
import pandas as pd
from wearable_htad.features.pipeline import process_accelerometer_file
from wearable_htad.features.pipeline import build_full_feature_dataset
# Set project root if needed
project_root = Path().resolve().parent
raw_folder = project_root / "data" / "htad" / "user1" / "raw" 

window_duration = 2.5
sampling_rate = 1000 / 35  # ms per sample ≈ 28.57 Hz
window_size = int(sampling_rate * window_duration)
overlap = 0.5

full_df = build_full_feature_dataset(
    folder=raw_folder,
    window_size=window_size,
    overlap=overlap
)

In [10]:
full_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 894 entries, 0 to 893
Data columns (total 22 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   x_mean     894 non-null    float64
 1   x_std      894 non-null    float64
 2   x_min      894 non-null    float64
 3   x_max      894 non-null    float64
 4   x_energy   894 non-null    float64
 5   x_entropy  894 non-null    float64
 6   y_mean     894 non-null    float64
 7   y_std      894 non-null    float64
 8   y_min      894 non-null    float64
 9   y_max      894 non-null    float64
 10  y_energy   894 non-null    float64
 11  y_entropy  894 non-null    float64
 12  z_mean     894 non-null    float64
 13  z_std      894 non-null    float64
 14  z_min      894 non-null    float64
 15  z_max      894 non-null    float64
 16  z_energy   894 non-null    float64
 17  z_entropy  894 non-null    float64
 18  sma        894 non-null    float64
 19  activity   894 non-null    object 
 20  sensor    

In [25]:
features_df = process_accelerometer_file(file_path, window_size=window_size, overlap=overlap)
features_df


,x_mean,x_std,x_min,x_max,x_energy,x_entropy,y_mean,y_std,y_min,y_max,...,z_mean,z_std,z_min,z_max,z_energy,z_entropy,sma,activity,sensor,timestamp
0,-0.350930,0.316921,-1.022949,0.097412,0.223590,1.555917,0.854406,0.240001,0.168945,1.257568,...,-0.164238,0.142978,-0.433594,0.227051,0.047417,1.589872,1.417020,watch_tv,acc,1468360517106
1,-0.116517,0.507975,-1.022949,0.703613,0.271615,2.097769,0.816066,0.260281,0.168945,1.287598,...,0.017255,0.220599,-0.433594,0.548096,0.048962,2.135733,1.430485,watch_tv,acc,1468360517106
2,0.294296,0.171590,-0.689941,0.703613,0.116053,1.217211,0.864894,0.264319,0.092529,1.424805,...,0.146921,0.262758,-0.175781,0.818115,0.090627,2.047578,1.389917,watch_tv,acc,1468360517106
3,0.307504,0.029305,0.235596,0.404541,0.095417,1.518579,0.888160,0.233290,0.092529,1.424805,...,0.043856,0.263024,-0.131592,0.818115,0.071105,1.030378,1.360338,watch_tv,acc,1468360517106
4,0.305465,0.003639,0.294678,0.312500,0.093322,1.868278,0.951853,0.007098,0.917480,0.972656,...,-0.077503,0.005093,-0.100586,-0.061279,0.006033,1.508349,1.334820,watch_tv,acc,1468360517106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,-0.473224,0.344898,-1.078125,0.344727,0.342895,1.178920,0.633583,0.232839,-0.236816,1.121582,...,0.434357,0.181042,-0.093750,0.749512,0.221442,1.347431,1.642520,watch_tv,acc,1468360517106
99,-0.620262,0.010188,-0.650146,-0.592773,0.384828,1.929412,0.639693,0.009980,0.612549,0.670654,...,0.463234,0.006919,0.451416,0.485352,0.214634,1.913107,1.723189,watch_tv,acc,1468360517106
100,-0.514576,0.197545,-0.632080,-0.062256,0.303813,0.854605,0.641887,0.048547,0.536133,0.869629,...,0.532402,0.151565,0.398682,0.928467,0.306424,1.011061,1.688865,watch_tv,acc,1468360517106
101,-0.233870,0.251388,-0.629883,0.084961,0.117891,1.694176,0.628050,0.074289,0.386719,0.869629,...,0.707513,0.182884,0.398682,1.188965,0.534021,1.615144,1.578125,watch_tv,acc,1468360517106


In [28]:
from wearable_htad.features.pipeline import build_full_feature_dataset

raw_folder = project_root / "data" / "htad" / "user1" / "raw"

full_df = build_full_feature_dataset(
    folder=raw_folder,
    window_size=window_size,
    overlap=overlap
)

full_df.head()



ImportError: cannot import name 'build_full_feature_dataset' from 'wearable_htad.features.pipeline' (/Users/johanneskeisers/Documents/GitHub/wearable-activity-htad/src/wearable_htad/features/pipeline.py)

In [4]:
from wearable_htad.features.extract import extract_features_from_window

# Just grab the first 128 rows (≈2.56s if 50Hz)
window = df.iloc[:128]
features = extract_features_from_window(window)
features


x_mean      -0.056181
x_std        0.415404
x_min       -1.022949
x_max        0.703613
x_energy     0.175717
x_entropy    1.972814
y_mean       0.847902
y_std        0.263069
y_min        0.092529
y_max        1.424805
y_energy     0.788143
y_entropy    1.788985
z_mean      -0.000929
z_std        0.275407
z_min       -0.433594
z_max        0.818115
z_energy     0.075850
z_entropy    1.899229
sma          1.406977
dtype: float64

In [ ]:
df = load_features_csv(data_path)
df_acc = load_accelerometer_csv(data_path)
df.head()

In [6]:
sys.path

['/opt/anaconda3/lib/python312.zip',
 '/opt/anaconda3/lib/python3.12',
 '/opt/anaconda3/lib/python3.12/lib-dynload',
 '',
 '/Users/johanneskeisers/Documents/GitHub/wearable-activity-htad/.venv/lib/python3.12/site-packages',
 '/Users/johanneskeisers/Documents/GitHub/wearable-activity-htad/src']